In [11]:
using JLD2
using Plots
using StatsBase
using LaTeXStrings
using Glob

In [21]:
data1 = jldopen("../data_archive/q_0p10_data/collated_results.jld2")["results"]
data2 = jldopen("../data_archive/no_unitary/collated_results.jld2")["results"]

Dict{NTuple{5, Any}, Tuple{Dict{Symbol, Vector{ComplexF64}}, Dict{Symbol, Vector{ComplexF64}}, Int64, Vector{Symbol}, Float64}} with 334 entries:
  (24, 50, 0.0, 0.7, 0.05) => (Dict(:κEA=>[24.0+0.0im, 24.0+0.0im, 24.0+0.0im, …
  (16, 34, 0.5, 0.7, 0.1)  => (Dict(:κEA=>[16.0+0.0im, 16.0+0.0im, 16.0+0.0im, …
  (2, 6, 0.2, 0.7, 0.1)    => (Dict(:κEA=>[2.0+0.0im, 2.0+0.0im, 2.0+0.0im, 1.1…
  (16, 34, 0.9, 0.7, 0.05) => (Dict(:κEA=>[16.0+0.0im, 16.0+0.0im, 16.0+0.0im, …
  (40, 82, 0.36, 0.7, 0.1) => (Dict(:κEA=>[40.0+0.0im, 40.0+0.0im, 40.0+0.0im, …
  (16, 34, 0.3, 0.7, 0.0)  => (Dict(:κEA=>[16.0+0.0im, 16.0+0.0im, 16.0+0.0im, …
  (32, 66, 0.1, 0.7, 0.1)  => (Dict(:κEA=>[32.0+0.0im, 32.0+0.0im, 32.0+0.0im, …
  (32, 66, 0.7, 0.7, 0.05) => (Dict(:κEA=>[32.0+0.0im, 32.0+0.0im, 32.0+0.0im, …
  (32, 66, 1.0, 0.7, 0.1)  => (Dict(:κEA=>[32.0+0.0im, 32.0+0.0im, 32.0+0.0im, …
  (16, 34, 1.0, 0.7, 0.45) => (Dict(:κEA=>[16.0+0.0im, 16.0+0.0im, 16.0+0.0im, …
  (32, 66, 0.5, 0.7, 0.05) => (Dict(:κEA=>[3

In [32]:
L = 8
[(key, data1[key][3]) for key in keys(data1) if key[1] == L]

11-element Vector{Tuple{Tuple{Int64, Int64, Float64, Float64, Float64}, Int64}}:
 ((8, 18, 0.1, 0.7, 0.1), 1000)
 ((8, 18, 1.0, 0.7, 0.1), 1000)
 ((8, 18, 0.2, 0.7, 0.1), 1000)
 ((8, 18, 0.7, 0.7, 0.1), 1000)
 ((8, 18, 0.5, 0.7, 0.1), 1000)
 ((8, 18, 0.3, 0.7, 0.1), 1000)
 ((8, 18, 0.9, 0.7, 0.1), 1000)
 ((8, 18, 0.4, 0.7, 0.1), 1000)
 ((8, 18, 0.6, 0.7, 0.1), 1000)
 ((8, 18, 0.8, 0.7, 0.1), 1000)
 ((8, 18, 0.0, 0.7, 0.1), 1000)

In [ ]:
# missing (q = 0.1, delta = 0.7):
# L = 40, lambda = 0.2, 0.3, 0.6, 40 samples
# L = 32, lambda = 0.8, 50 samples
# L = 24, lambda = 0.3, 200 samples
# L = 24, lambda = 0.9, 100 samples
# L = 16, lambda = 0.1, 100 samples

In [56]:
[(key, data2[key][3]) for key in keys(data2) if key[1] == 24 && key[5] == 0.1 && key[3] < 0.5] 

14-element Vector{Tuple{Tuple{Int64, Int64, Float64, Float64, Float64}, Int64}}:
 ((24, 50, 0.38, 0.7, 0.1), 1000)
 ((24, 50, 0.2, 0.7, 0.1), 1000)
 ((24, 50, 0.34, 0.7, 0.1), 1000)
 ((24, 50, 0.0, 0.7, 0.1), 2000)
 ((24, 50, 0.36, 0.7, 0.1), 1000)
 ((24, 50, 0.1, 0.7, 0.1), 1000)
 ((24, 50, 0.3, 0.7, 0.1), 1000)
 ((24, 50, 0.39, 0.7, 0.1), 1000)
 ((24, 50, 0.35, 0.7, 0.1), 700)
 ((24, 50, 0.31, 0.7, 0.1), 1000)
 ((24, 50, 0.32, 0.7, 0.1), 1000)
 ((24, 50, 0.4, 0.7, 0.1), 1000)
 ((24, 50, 0.37, 0.7, 0.1), 900)
 ((24, 50, 0.33, 0.7, 0.1), 900)

In [ ]:
# missing (q = 0.1, delta = 0.7):
# L = 40, lambda = 0.35, 40 samples
# L = 32, lambda = 0.39 -- 150
#                  0.35 -- 50
#                  0.31 -- 150
#                  0.33 -- 50
#                  0.38 -- 150
#                  0.34 -- 50
#                  0.36 -- 50
#                  0.71 -- 200
#                  0.74 -- 150
#                  0.78 -- 100
#                  0.72 -- 150
#                  0.73 -- 50
#                  0.75 -- 350
#                  0.76 -- 350
#                  0.77 -- 200
# L = 24, lambda = 0.35 -- 300
#                  0.37 -- 100
#                  0.33 -- 100
#                  0.74 -- 100
#                  0.78 -- 100
#

In [60]:
[(key, data2[key][3]) for key in keys(data2) if key[1] == 40 && key[5] == 0.0] 

11-element Vector{Tuple{Tuple{Int64, Int64, Float64, Float64, Float64}, Int64}}:
 ((40, 82, 0.2, 0.7, 0.0), 1000)
 ((40, 82, 0.0, 0.7, 0.0), 960)
 ((40, 82, 0.1, 0.7, 0.0), 1000)
 ((40, 82, 1.0, 0.7, 0.0), 1000)
 ((40, 82, 0.3, 0.7, 0.0), 1000)
 ((40, 82, 0.8, 0.7, 0.0), 1000)
 ((40, 82, 0.7, 0.7, 0.0), 1000)
 ((40, 82, 0.5, 0.7, 0.0), 280)
 ((40, 82, 0.4, 0.7, 0.0), 1000)
 ((40, 82, 0.6, 0.7, 0.0), 1000)
 ((40, 82, 0.9, 0.7, 0.0), 1000)

In [ ]:
# missing (can run these PURE!) (q = 0.0, delta = 0.7)
# L = 40 lambda 0.0 -- 40
#               0.5 -- 1000 (just rerun... and maybe keep separate lol)

In [53]:
[(key, data2[key][3]) for key in keys(data2) if key[1] == 40 && key[5] == 0.05] 

11-element Vector{Tuple{Tuple{Int64, Int64, Float64, Float64, Float64}, Int64}}:
 ((40, 82, 0.7, 0.7, 0.05), 840)
 ((40, 82, 0.5, 0.7, 0.05), 720)
 ((40, 82, 0.4, 0.7, 0.05), 1000)
 ((40, 82, 0.6, 0.7, 0.05), 40)
 ((40, 82, 0.9, 0.7, 0.05), 1000)
 ((40, 82, 0.2, 0.7, 0.05), 1000)
 ((40, 82, 0.0, 0.7, 0.05), 1000)
 ((40, 82, 0.1, 0.7, 0.05), 1000)
 ((40, 82, 1.0, 0.7, 0.05), 1000)
 ((40, 82, 0.3, 0.7, 0.05), 1000)
 ((40, 82, 0.8, 0.7, 0.05), 1000)

In [65]:
[(key, data2[key][3]) for key in keys(data2) if key[1] == 24 && key[3] == 0.0] 

11-element Vector{Tuple{Tuple{Int64, Int64, Float64, Float64, Float64}, Int64}}:
 ((24, 50, 0.0, 0.7, 0.05), 2000)
 ((24, 50, 0.0, 0.7, 0.35), 500)
 ((24, 50, 0.0, 0.7, 0.2), 1000)
 ((24, 50, 0.0, 0.7, 0.1), 2000)
 ((24, 50, 0.0, 0.7, 0.15), 1000)
 ((24, 50, 0.0, 0.7, 0.4), 1000)
 ((24, 50, 0.0, 0.7, 0.0), 2000)
 ((24, 50, 0.0, 0.7, 0.45), 1000)
 ((24, 50, 0.0, 0.7, 0.3), 600)
 ((24, 50, 0.0, 0.7, 0.5), 1000)
 ((24, 50, 0.0, 0.7, 0.25), 700)

In [55]:
[(key, data2[key][3]) for key in keys(data2) if key[1] == 40 && key[3] == 1.0] 

3-element Vector{Tuple{Tuple{Int64, Int64, Float64, Float64, Float64}, Int64}}:
 ((40, 82, 1.0, 0.7, 0.1), 1000)
 ((40, 82, 1.0, 0.7, 0.0), 1000)
 ((40, 82, 1.0, 0.7, 0.05), 1000)